In [ ]:
from aoc2024 import load_example, load_input

In [ ]:
garden_plots = [[plot for plot in row] for row in load_input(12).splitlines()]
garden_plots

In [ ]:
DIRECTIONS = [
    (-1, 0), #UP
    (0, -1), #LEFT
    (1, 0), #DOWN
    (0, 1), #RIGHT
]

In [ ]:
from collections import deque, defaultdict
from tqdm import tqdm

def is_within_bounds(y: int, x: int, garden_plots: list[list[str]]) -> bool:
    return 0 <= y < len(garden_plots) and 0 <= x < len(garden_plots[0])

def find_regions(garden_plots: list[list[str]]):
    visited_coordinates = set()

    def is_part_of_region(coordinates: tuple[int, int], plant: str) -> bool:
        y, x = coordinates
        if is_within_bounds(y, x, garden_plots):
            return garden_plots[y][x] == plant

        return False
    
    regions = defaultdict(list)

    total_y = len(garden_plots)
    total_x = len(garden_plots[0])

    with tqdm(total=total_y * total_x) as pbar:
        for y in range(total_y):
            for x in range(total_x):
                if (position := (y, x)) not in visited_coordinates:
                    plant = garden_plots[y][x]
                    region = set()
                    to_check = deque([position])
                    visited_coordinates.add(position)
                    
                    while len(to_check) > 0:
                        current_position = to_check.popleft()
                        visited_coordinates.add(current_position)
                        region.add(current_position)
                        new_coordinates = [(current_position[0] + y, current_position[1] + x) for (y, x) in DIRECTIONS]
                        correct_new_coordinates = [c for c in new_coordinates if is_part_of_region(c, plant) and c not in visited_coordinates]
                        to_check.extend(correct_new_coordinates)
                        visited_coordinates.update(correct_new_coordinates)

                    regions[plant].append(region)
                    pbar.update(1)
        
    return regions


In [ ]:
regions = find_regions(garden_plots)
regions

In [ ]:
def area(region: set[tuple[int, int]]) -> int:
    return len(region)


def perimiter(region: set[tuple[int, int]], plant: str, garden_plots: list[list[str]]) -> int:
    total_perimiter = 0

    for (y, x) in region:
        neighbor_coordinates = [(y + y_d, x + x_d) for (y_d, x_d) in DIRECTIONS]
        not_in_region = [c for c in neighbor_coordinates if c not in region]
        total_perimiter += len(not_in_region)
    return total_perimiter

In [ ]:
total_price = 0
for plant, plant_regions in regions.items():
    
    for region in plant_regions:
        a = area(region)
        p = perimiter(region, plant, garden_plots)
        # print(f"region of {plant} plants with price {a} * {p} = {a * p}.")
        total_price += a * p
print(total_price)